Načítanie knihovien

In [1]:
import pandas as pd
from scipy.stats import chi2_contingency, ttest_ind

Načítanie dataframe

In [2]:
df = pd.read_pickle("accidents.pkl.gz")
df

,p1,p36,p37,p2a,weekday(p2a),p2b,p6,p7,p8,p9,...,l,n,o,p,q,r,s,t,p5a,region
0,002100160001,4,-1,2016-01-01,5,55,1,1,0,2,...,,711403,,Souhlasnýsesměremúseku,Pomalý,554782,451622,GN_V0.1UIR-ADR_410,1,PHA
1,002100160002,4,-1,2016-01-01,5,130,1,3,0,2,...,,,,,,-1,-1,,1,PHA
2,002100160003,5,-1,2016-01-01,5,100,1,2,0,2,...,,,,,,-1,-1,,1,PHA
3,002100160004,6,-1,2016-01-01,5,120,9,0,0,2,...,,,,,,-1,-1,,1,PHA
4,002100160005,6,-1,2016-01-01,5,2560,2,0,0,2,...,,,,,,-1,-1,,1,PHA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572929,190906210457,6,-1,2021-08-30,1,506,2,0,0,2,...,,2611387,,,Pomalý,560359,-1,GN_V0.1UIR-ADR_410,1,KVK
572930,190906210458,6,-1,2021-08-30,1,752,2,0,0,2,...,,2541503,,Opačnýkesměruúseku,Pomalý,560383,550426,GN_V0.1UIR-ADR_410,1,KVK
572931,190906210459,3,2099,2021-08-30,1,1505,1,3,0,2,...,,,,,,-1,-1,,1,KVK
572932,190906210460,3,2099,2021-08-31,2,1245,0,0,0,1,...,2099,199764,,Opačnýkesměruúseku,Pomalý,560286,553654,GN_V0.1UIR-ADR_410,1,KVK


# Hypotéza 1
Na silnicích první třídy se pri nehodách umíralo se stejnou pravdepodobností jako na silnicích tretí třídy.

Filtrovanie údajov na základe zadania a vytvorenie kontingenčnej tabuľky

In [3]:
df["fatal"] = df["p13a"] > 0
filtered = df[df["p36"].isin([1, 3])]
ct = pd.crosstab(index=filtered["fatal"], columns=filtered["p36"])
ct

p36,1,3
fatal,,
False,78618,73352
True,911,448


Výpočet pomocou χ2 testu

In [4]:
res = chi2_contingency(ct)
res

(125.72070150000258,
 3.5395243450138555e-29,
 1,
 array([[78824.11109444, 73145.88890556],
        [  704.88890556,   654.11109444]]))

Porovnanie výsledkov

In [5]:
ct - res[3]

p36,1,3
fatal,,
False,-206.111094,206.111094
True,206.111094,-206.111094


## Hypotéza 1 - záver
Pravdepodobnost 3.5 * 10^(-29) je oveľa menšia ako 0.05. Z tohto dôvodu považujem nulovú hypotézu za nepravdivú a vyhlasujem, že existuje veľká korelácia medzi pravdepodobnosťou smrteľných nehôd na cestách prvej a tretej triedy. Tiež vidíme, že cesty prvej triedy si celkovo vyžiadali viac smrteľných zranení.

# Hypotéza 2
Při nehodách vozidel značky Škoda je škoda na vozidle nižší než při nehodách vozidel Audi.

Znovu inicializujem hodnoty

In [6]:
df2 = pd.read_pickle("accidents.pkl.gz")

Filtrovanie údajov na základe zadania a vytvorenie kontingenčnej tabuľky

In [7]:
filtered = df2[df2["p45a"].isin([2, 39])]
ct = pd.crosstab(index=filtered["p53"], columns=filtered["p45a"])
ct

p45a,2,39
p53,,
0,764,8203
1,12,240
2,1,74
3,1,13
4,0,7
...,...,...
9000,4,2
10000,10,8
12000,2,0


Zistím celkové škody pre jednolivé značky áut

In [8]:
print("Audi: " + str(sum(ct[2])))
print("Skoda: " + str(sum(ct[39])))

Audi: 11445
Skoda: 118379


Použijem t-test

In [9]:
t_statistic, p_value = ttest_ind(ct[2], ct[39])
p_value

0.0007078755607522958

## Hypotéza 2 - záver
Pravdepodobnosť 0.0007 je menšia ako 0.05. Z tohto dôvodu považujem nulovú hypotézu za nepravdivú. Škoda na vozidlách značky Škoda je oveľa vyššia ako na vodizidlách značky Audi. 